In [113]:
import requests
from bs4 import BeautifulSoup
class Page:
    def __init__(self, site=None, ticker_idx=0, params=None):
        self.tickers = ['4PX-DPAR', '2PX-DPAR', '1PX-DPAR', '5PX-DPAR']
        # liste des options euronext CAC40 :
        # https://derivatives.euronext.com/en/equity-index-derivatives/contract-list?quicktabs_309=4#quicktabs-309
        self.site="https://derivatives.euronext.com/fr/products/index-options" if not site else site
        self.ticker=self.tickers[ticker_idx]
        self.params="Class_type=0&Class_symbol=&Class_exchange=&ex=&ps=15&md=11-2018" if not params else params
    def fetch(self):
        requete = requests.get("{}/{}?{}".format(site, ticker, params))
        page = requete.content
        self.soup = BeautifulSoup(page)
        self.page_title = self.soup.find("h1", {"class": "title"}).text
        return self.soup
    def scrap_elements(self):
        div = self.soup.find("div", {"class": "call-put-table"})
        trs = div.find_all("tr")
        elmts = {}
        for tr in trs[3:-1]:
            tds = tr.find_all('td')
            c = Option('Call', strike=tds[7].text, achat=tds[4].text, vente=tds[5].text)
            p = Option('Put', strike=tds[7].text, achat=tds[9].text, vente=tds[10].text)
            if c.achat != '-':
                elmts[float(c.strike)]={'call':c, 'put':p}
        return elmts
p = Page()
s = p.fetch()

In [114]:
p.page_title

'CAC 40 INDEX WEEKLY OPTION (1PX 2PX 4PX 5PX)'

In [103]:
class Option:
    def __init__(self, cat='Call', strike=0, achat=0, vente=0, premium=0, multiplier=1):
        self.cat = cat
        self.strike = strike
        self.achat = achat
        self.vente = vente
        self.premium = premium
        self.multiplier = multiplier
    def __repr__(self):
        #return '{}({}, {}, {})'.format(self.cat, self.strike, self.achat, self.vente)
        return '{}({})'.format(self.cat, self.strike)
    def __str__(self):
        return '{}({}) long:{}, short:{}'.format(self.cat, self.strike, self.achat, self.vente)


In [92]:
o=Option(cat='Call', strike=5200)
'{} bid:{}, ask:{}'.format(o, o.achat, o.vente)
str(o)

'Call(5200) long:0, short:0'

In [104]:
options = p.scrap_elements()
options

{4900.0: {'call': Call(4900.00), 'put': Put(4900.00)},
 4950.0: {'call': Call(4950.00), 'put': Put(4950.00)},
 5000.0: {'call': Call(5000.00), 'put': Put(5000.00)},
 5050.0: {'call': Call(5050.00), 'put': Put(5050.00)},
 5075.0: {'call': Call(5075.00), 'put': Put(5075.00)},
 5100.0: {'call': Call(5100.00), 'put': Put(5100.00)},
 5125.0: {'call': Call(5125.00), 'put': Put(5125.00)},
 5150.0: {'call': Call(5150.00), 'put': Put(5150.00)},
 5175.0: {'call': Call(5175.00), 'put': Put(5175.00)},
 5200.0: {'call': Call(5200.00), 'put': Put(5200.00)},
 5250.0: {'call': Call(5250.00), 'put': Put(5250.00)},
 5300.0: {'call': Call(5300.00), 'put': Put(5300.00)}}

In [105]:
str(options[4900]['call'])

'Call(4900.00) long:212.40, short:219.40'